In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from collections import Counter
pd.set_option('display.max_columns', None)

In [2]:
# Writer which will be used until the end
writer = pd.ExcelWriter("sct_processed_data.xlsx", engine='xlsxwriter')

In [3]:
# Load the data into a df and remove the junk headers
df = pd.read_csv("Adverb++Locations_February+20,+2021_06.38.csv")
df = df.iloc[2:]
finished_df = df[df['Progress'] == '100']

# Remove any participants that should be dropped
finished_df = finished_df[finished_df["Drop"] != 'x']
finished_df = finished_df.apply(pd.to_numeric, errors='ignore')
finished_df.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,SCT Trial_0_GROUP,SCT Trial_1_GROUP,SCT Trial_2_GROUP,SCT Trial_3_GROUP,SCT Trial_0_1_RANK,SCT Trial_0_2_RANK,SCT Trial_0_3_RANK,SCT Trial_0_11_RANK,SCT Trial_1_1_RANK,SCT Trial_1_2_RANK,SCT Trial_1_3_RANK,SCT Trial_1_11_RANK,SCT Trial_2_1_RANK,SCT Trial_2_2_RANK,SCT Trial_2_3_RANK,SCT Trial_2_11_RANK,SCT Trial_3_1_RANK,SCT Trial_3_2_RANK,SCT Trial_3_3_RANK,SCT Trial_3_11_RANK,SCT Trial 2_0_GROUP,SCT Trial 2_1_GROUP,SCT Trial 2_2_GROUP,SCT Trial 2_3_GROUP,SCT Trial 2_0_1_RANK,SCT Trial 2_0_2_RANK,SCT Trial 2_0_3_RANK,SCT Trial 2_0_11_RANK,SCT Trial 2_1_1_RANK,SCT Trial 2_1_2_RANK,SCT Trial 2_1_3_RANK,SCT Trial 2_1_11_RANK,SCT Trial 2_2_1_RANK,SCT Trial 2_2_2_RANK,SCT Trial 2_2_3_RANK,SCT Trial 2_2_11_RANK,SCT Trial 2_3_1_RANK,SCT Trial 2_3_2_RANK,SCT Trial 2_3_3_RANK,SCT Trial 2_3_11_RANK,SCT0_A-_aqui_0_GROUP,SCT0_A-_aqui_1_GROUP,SCT0_A-_aqui_2_GROUP,SCT0_A-_aqui_3_GROUP,SCT0_A-_aqui_0_1_RANK,SCT0_A-_aqui_0_2_RANK,SCT0_A-_aqui_0_3_RANK,SCT0_A-_aqui_0_4_RANK,SCT0_A-_aqui_1_1_RANK,SCT0_A-_aqui_1_2_RANK,SCT0_A-_aqui_1_3_RANK,SCT0_A-_aqui_1_4_RANK,SCT0_A-_aqui_2_1_RANK,SCT0_A-_aqui_2_2_RANK,SCT0_A-_aqui_2_3_RANK,SCT0_A-_aqui_2_4_RANK,SCT0_A-_aqui_3_1_RANK,SCT0_A-_aqui_3_2_RANK,SCT0_A-_aqui_3_3_RANK,SCT0_A-_aqui_3_4_RANK,SCT1_A-_aqui_0_GROUP,SCT1_A-_aqui_1_GROUP,SCT1_A-_aqui_2_GROUP,SCT1_A-_aqui_3_GROUP,SCT1_A-_aqui_0_1_RANK,SCT1_A-_aqui_0_2_RANK,SCT1_A-_aqui_0_3_RANK,SCT1_A-_aqui_0_4_RANK,SCT1_A-_aqui_1_1_RANK,SCT1_A-_aqui_1_2_RANK,SCT1_A-_aqui_1_3_RANK,SCT1_A-_aqui_1_4_RANK,SCT1_A-_aqui_2_1_RANK,SCT1_A-_aqui_2_2_RANK,SCT1_A-_aqui_2_3_RANK,SCT1_A-_aqui_2_4_RANK,SCT1_A-_aqui_3_1_RANK,SCT1_A-_aqui_3_2_RANK,SCT1_A-_aqui_3_3_RANK,SCT1_A-_aqui_3_4_RANK,SCT0_A-_afuera_0_GROUP,SCT0_A-_afuera_1_GROUP,SCT0_A-_afuera_2_GROUP,SCT0_A-_afuera_3_GROUP,SCT0_A-_afuera_0_1_RANK,SCT0_A-_afuera_0_2_RANK,SCT0_A-_afuera_0_3_RANK,SCT0_A-_afuera_0_4_RANK,SCT0_A-_afuera_1_1_RANK,SCT0_A-_afuera_1_2_RANK,SCT0_A-_afuera_1_3_RANK,SCT0_A-_afuera_1_4_RANK,SCT0_A-_afuera_2_1_RANK,SCT0_A-_afuera_2_2_RANK,SCT0_A-_afuera_2_3_RANK,SCT0_A-_afuera_2_4_RANK,SCT0_A-_afuera_3_1_RANK,SCT0_A-_afuera_3_2_RANK,SCT0_A-_afuera_3_3_RANK,SCT0_A-_afuera_3_4_RANK,SCT1_A-_afuera_0_GROUP,SCT1_A-_afuera_1_GROUP,SCT1_A-_afuera_2_GROUP,SCT1_A-_afuera_3_GROUP,SCT1_A-_afuera_0_1_RANK,SCT1_A-_afuera_0_2_RANK,SCT1_A-_afuera_0_3_RANK,SCT1_A-_afuera_0_4_RANK,SCT1_A-_afuera_1_1_RANK,SCT1_A-_afuera_1_2_RANK,SCT1_A-_afuera_1_3_RANK,SCT1_A-_afuera_1_4_RANK,SCT1_A-_afuera_2_1_RANK,SCT1_A-_afuera_2_2_RANK,SCT1_A-_afuera_2_3_RANK,SCT1_A-_afuera_2_4_RANK,SCT1_A-_afuera_3_1_RANK,SCT1_A-_afuera_3_2_RANK,SCT1_A-_afuera_3_3_RANK,SCT1_A-_afuera_3_4_RANK,SCT0_A-_cerca_0_GROUP,SCT0_A-_cerca_1_GROUP,SCT0_A-_cerca_2_GROUP,SCT0_A-_cerca_3_GROUP,SCT0_A-_cerca_0_1_RANK,SCT0_A-_cerca_0_2_RANK,SCT0_A-_cerca_0_3_RANK,SCT0_A-_cerca_0_4_RANK,SCT0_A-_cerca_1_1_RANK,SCT0_A-_cerca_1_2_RANK,SCT0_A-_cerca_1_3_RANK,SCT0_A-_cerca_1_4_RANK,SCT0_A-_cerca_2_1_RANK,SCT0_A-_cerca_2_2_RANK,SCT0_A-_cerca_2_3_RANK,SCT0_A-_cerca_2_4_RANK,SCT0_A-_cerca_3_1_RANK,SCT0_A-_cerca_3_2_RANK,SCT0_A-_cerca_3_3_RANK,SCT0_A-_cerca_3_4_RANK,SCT1_A-_cerca_0_GROUP,SCT1_A-_cerca_1_GROUP,SCT1_A-_cerca_2_GROUP,SCT1_A-_cerca_3_GROUP,SCT1_A-_cerca_0_1_RANK,SCT1_A-_cerca_0_2_RANK,SCT1_A-_cerca_0_3_RANK,SCT1_A-_cerca_0_4_RANK,SCT1_A-_cerca_1_1_RANK,SCT1_A-_cerca_1_2_RANK,SCT1_A-_cerca_1_3_RANK,SCT1_A-_cerca_1_4_RANK,SCT1_A-_cerca_2_1_RANK,SCT1_A-_cerca_2_2_RANK,SCT1_A-_cerca_2_3_RANK,SCT1_A-_cerca_2_4_RANK,SCT1_A-_cerca_3_1_RANK,SCT1_A-_cerca_3_2_RANK,SCT1_A-_cerca_3_3_RANK,SCT1_A-_cerca_3_4_RANK,SCT0_A-_lejos_0_GROUP,SCT0_A-_lejos_1_GROUP,SCT0_A-_lejos_2_GROUP,SCT0_A-_lejos_3_GROUP,SCT0_A-_lejos_0_1_RANK,SCT0_A-_lejos_0_2_RANK,SCT0_A-_lejos_0_3_RANK,SCT0_A-_lejos_0_4_RANK,SCT0_A-_lejos_1_1_RANK,SCT0_A-_lejos_1_2_RANK,

In [4]:
# Need to go through all the rank columns and find the word order
# The coding scheme has the first number after the adverb_ is the position (starting at zero). 
# 0 means that it is the first word placed
# The next number after the underscore is the word type which uses the following key
# 1 - Subject
# 2 - Verb
# 3 - Object
# 4 - Adverb

# Need ways to get the question_id, the position, the word type
def get_question_id(column_name):
    qid_list = column_name.split("_")[:3]
    question_id = "_".join(qid_list)
    
    # Qualtrics doesn't like how long aproximadamente is, so it was outputted as aproximadame
    # We fix this by replacing aproximadame with aproximadamente
    if question_id.split("_")[-1] == "aproximadame":
        question_id = question_id.replace("aproximadame", "aproximadamente")
    
    return question_id

def get_position(column_name):
    return column_name.split("_")[3]

def get_word_type(column_name):
    word_type_key = {
        "1": "S",
        "2": "V",
        "3": "O",
        "4": "Adv"
    }
    
    word_type_value = column_name.split("_")[4]
    return word_type_key[word_type_value]

def group_columns_with_same_question_id(column_names):
    grouped_col_names_dict = defaultdict(list)
    
    for col_name in column_names:
        question_id = get_question_id(col_name)
        grouped_col_names_dict[question_id].append(col_name)
        
    # Convert the dictionary to a list
    grouped_col_names = [sorted(grouped_col_names_dict[k]) for k in grouped_col_names_dict if "SCT" in k]
    
    return grouped_col_names


def label_group(row):
    if not pd.isna(row["N1"]):
        return "Native"
    if int(row['SC0']) < 30:
        return "Beginner"
    if int(row['SC0']) < 40:
        return "Intermediate"
    return "Advanced"


def find_word_order(grouped_df):
    # a new df which has columns of the question id
    qid = get_question_id(grouped_df.columns[0])
    number_of_partipants = len(grouped_df.index)
    
    word_order_df = pd.DataFrame(index=grouped_df.index, columns=[qid], dtype=np.str)
    word_order_df.index.name = "PID"
    word_order_df[qid] = ""
    
    for column in grouped_df:
        if column in ["PID"]:
            continue
            
        word_type = get_word_type(column)
        position = get_position(column)
        
        # If the cell has a 1 in it, then you can add the word type (S or V or O or Adv)
        for index, row in grouped_df.iterrows():
            if not pd.isnull(row[column]):
                if pd.isnull(word_order_df.loc[index][qid]):
                    word_order_df.loc[index][qid] = word_type
                else:
                    word_order_df.loc[index][qid] += word_type
                
    return word_order_df

# Now we actually make the df we want
def make_word_order_df(original_df):
    grouped_cols = group_columns_with_same_question_id(list(original_df.columns.values))

    dfs_to_merge = []
    for group in grouped_cols:
        # Now we find the word order
        grouped_df = original_df[group]
        dfs_to_merge.append(find_word_order(grouped_df))

    word_order_df = pd.concat(dfs_to_merge, axis=1).reset_index()
    return word_order_df

filter_cols = [col for col in finished_df if ("SCT" in col and "trial" not in col.lower() and "GROUP" not in col)]

word_order_df = make_word_order_df(finished_df[filter_cols])
word_order_df.head()

,PID,SCT0_A-_aqui,SCT1_A-_aqui,SCT0_A-_afuera,SCT1_A-_afuera,SCT0_A-_cerca,SCT1_A-_cerca,SCT0_A-_lejos,SCT1_A-_lejos,SCT0_B-_casi,SCT1_B-_casi,SCT0_B+_aproximadamente,SCT1_B+_aproximadamente,SCT0_B-_apenas,SCT1_B-_apenas,SCT0_B+_solamente,SCT1_B+_solamente,SCT0_C+_facilmente,SCT1_C+_facilmente,SCT0_C-_bien,SCT1_C-_bien,SCT0_C-_mal,SCT1_C-_mal,SCT0_C+_precisamente,SCT1_C+_precisamente,SCT0_D-_mucho,SCT1_D-_mucho,SCT0_D+_mayormente,SCT1_D+_mayormente,SCT0_D-_poco,SCT1_D-_poco,SCT0_D+_minimamente,SCT1_D+_minimamente
0,4,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVAdvO,SVOAdv,SVAdvO,SVAdvO,SAdvVO,SAdvVO,SVOAdv,SAdvVO,SAdvVO,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SAdvVO,SVOAdv,SAdvVO,SVAdvO,SVAdvO,SVOAdv,SVOAdv,SAdvVO,SVOAdv,SVOAdv,SVOAdv
1,5,AdvVSO,AdvVSO,AdvVSO,AdvVSO,SOVAdv,AdvVSO,AdvVSO,AdvVSO,SVAdvO,SVAdvO,SOVAdv,SOVAdv,SVAdvO,SVAdvO,SVAdvO,SVAdvO,SOVAdv,SOVAdv,SOVAdv,SOVAdv,SAdvVO,SAdvVO,SVAdvO,SVAdvO,SOVAdv,SOVAdv,SOVAdv,SAdvVO,SAdvVO,SAdvVO,SOVAdv,SOVAdv
2,8,SVOAdv,AdvSVO,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVAdvO,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVAdvO,SVAdvO,SAdvVO,SAdvVO,SVOAdv,SVOAdv,SVOAdv,SVOAdv
3,11,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SAdvVO,SAdvVO,SAdvVO,SVOAdv,SAdvVO,SAdvVO,SAdvVO,SAdvVO,SVAdvO,SVOAdv,SVOAdv,SVOAdv,SVAdvO,SVAdvO,SVOAdv,SVAdvO,SVOAdv,SVOAdv,SAdvVO,SVAdvO,SAdvVO,SAdvVO,SVOAdv,SAdvVO
4,16,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVAdvO,SVOAdv,SVOAdv,SVOAdv,SAdvVO,SAdvVO,SAdvVO,SVOAdv,SAdvVO,SAdvVO,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SVOAdv,SAdvVO,SVOAdv,SVOAdv,SVAdvO,SVAdvO,SVOAdv,SAdvVO,AdvSVO,SVOAdv


In [5]:
qids = [col for col in word_order_df.columns if "SCT" in col]
df = word_order_df.melt(id_vars="PID", value_vars=qids, value_name="Word_Order").rename(columns={"variable": "QID"})
df

,PID,QID,Word_Order
0,4,SCT0_A-_aqui,SVOAdv
1,5,SCT0_A-_aqui,AdvVSO
2,8,SCT0_A-_aqui,SVOAdv
3,11,SCT0_A-_aqui,SVOAdv
4,16,SCT0_A-_aqui,SVOAdv
...,...,...,...
2171,108,SCT1_D+_minimamente,SAdvVO
2172,109,SCT1_D+_minimamente,SVOAdv
2173,110,SCT1_D+_minimamente,SAdvVO
2174,111,SCT1_D+_minimamente,SVOAdv


In [6]:
# Add the score column and N1 to check for natives
df = pd.merge(df, finished_df.reset_index()[["SC0", 'PID', "N1"]], on="PID")

# Add a group column and remove the score and N1 columns
df["Group"] = df.apply(lambda row: label_group(row), axis=1)
df.drop(["SC0", "N1"], axis=1, inplace=True)

# Add columns for the adverb type, mente, and adverb
def get_adverb_type(column_name):
    type_key = {
        "A": "Place",
        "B": "Extent",
        "C": "Manner",
        "D": "Quantity"
    }
    
    adverb_type_val = column_name.split("_")[1][0]
    return type_key[adverb_type_val]

def has_mente(column_name):
    return "Yes" if column_name.split("_")[1][1] == "+" else "No"

def get_adverb(column_name):
    return column_name.split("_")[-1]

df["Adverb_Class"] = df.apply(lambda row: get_adverb_type(row["QID"]), axis=1)
df["Mente"] = df.apply(lambda row: has_mente(row["QID"]), axis=1)
df["Adverb"] = df.apply(lambda row: get_adverb(row["QID"]), axis=1)
df.to_csv('sct_data.csv', index=False)

print(df['Word_Order'].value_counts())

# These make the ordering of the output nice
df['Group'] = pd.Categorical(df['Group'], ["Beginner", "Intermediate", "Advanced", "Native"])
df['Adverb'] = pd.Categorical(df['Adverb'], ['aqui', 'afuera', 'cerca', 'lejos', 'casi', 'aproximadamente', 
                      'apenas', 'solamente', 'facilmente', 'bien', 'mal', 
                      'precisamente', 'mucho', 'mayormente', 'poco', 'minimamente'])
df['Adverb_Class'] = pd.Categorical(df['Adverb_Class'], ['Place', 'Extent', 'Manner', 'Quantity'])
df['Word_Order'] = pd.Categorical(df['Word_Order'], ["AdvSVO", "SAdvVO", "SVAdvO", "SVOAdv"])

df

SVOAdv    1247
SAdvVO     460
SVAdvO     342
AdvSVO      57
SOVAdv      53
AdvVSO       9
SAdvOV       4
VSOAdv       2
AdvVOS       1
             1
Name: Word_Order, dtype: int64


,PID,QID,Word_Order,Group,Adverb_Class,Mente,Adverb
0,4,SCT0_A-_aqui,SVOAdv,Intermediate,Place,No,aqui
1,4,SCT1_A-_aqui,SVOAdv,Intermediate,Place,No,aqui
2,4,SCT0_A-_afuera,SVOAdv,Intermediate,Place,No,afuera
3,4,SCT1_A-_afuera,SVOAdv,Intermediate,Place,No,afuera
4,4,SCT0_A-_cerca,SVAdvO,Intermediate,Place,No,cerca
...,...,...,...,...,...,...,...
2171,112,SCT1_D+_mayormente,SVAdvO,Intermediate,Quantity,Yes,mayormente
2172,112,SCT0_D-_poco,SVOAdv,Intermediate,Quantity,No,poco
2173,112,SCT1_D-_poco,SVOAdv,Intermediate,Quantity,No,poco
2174,112,SCT0_D+_minimamente,SVOAdv,Intermediate,Quantity,Yes,minimamente


In [7]:
# Get the number of times each structure appeared for each question
struct_freq_df = pd.crosstab(index=df['Group'], columns=df['Word_Order'], normalize='index')
struct_freq_df.to_excel(writer, sheet_name='Struct Freqs')
struct_freq_df

Word_Order,AdvSVO,SAdvVO,SVAdvO,SVOAdv
Group,,,,
Beginner,0.028415,0.179235,0.102732,0.689617
Intermediate,0.021687,0.255422,0.113253,0.609639
Advanced,0.013393,0.223214,0.227679,0.535714
Native,0.034420,0.253623,0.271739,0.440217


In [8]:
# Get the structure frequencies with and without mente
mente_struct_freqs_df = pd.crosstab(index=[df['Mente'], df['Group']], columns=df['Word_Order'], normalize='index')
mente_struct_freqs_df.to_excel(writer, sheet_name='Mente Struct Freqs')
mente_struct_freqs_df

Word_Order            AdvSVO    SAdvVO    SVAdvO    SVOAdv
Mente Group                                               
No    Beginner      0.042105  0.156140  0.117544  0.684211
      Intermediate  0.023166  0.227799  0.138996  0.610039
      Advanced      0.021429  0.214286  0.207143  0.557143
      Native        0.052023  0.234104  0.260116  0.453757
Yes   Beginner      0.005797  0.217391  0.078261  0.698551
      Intermediate  0.019231  0.301282  0.070513  0.608974
      Advanced      0.000000  0.238095  0.261905  0.500000
      Native        0.004854  0.286408  0.291262  0.417476

In [9]:
# Get the placement per adverb
adverb_placement_df = pd.crosstab(index=[df['Adverb'], df['Group']], 
                                  columns=df['Word_Order'],
                                 normalize='index')
adverb_placement_df.to_excel(writer, sheet_name='Adverb Struct Freqs')
adverb_placement_df

Word_Order                  AdvSVO    SAdvVO    SVAdvO    SVOAdv
Adverb      Group                                               
aqui        Beginner      0.224138  0.000000  0.000000  0.775862
            Intermediate  0.153846  0.076923  0.000000  0.769231
            Advanced      0.142857  0.071429  0.142857  0.642857
            Native        0.451613  0.000000  0.000000  0.548387
afuera      Beginner      0.017241  0.000000  0.017241  0.965517
...                            ...       ...       ...       ...
poco        Native        0.000000  0.210526  0.552632  0.236842
minimamente Beginner      0.000000  0.241379  0.137931  0.620690
            Intermediate  0.038462  0.230769  0.038462  0.692308
            Advanced      0.000000  0.214286  0.071429  0.714286
            Native        0.000000  0.235294  0.382353  0.382353

[64 rows x 4 columns]

In [10]:
# Get the placement per adverb class
adv_class_placement_df = pd.crosstab(index=[df['Adverb_Class'], df['Group']], 
                                     columns=df['Word_Order'],
                                    normalize='index')
adv_class_placement_df.to_excel(writer, sheet_name='Adverb Class Struct Freqs')
adv_class_placement_df

Word_Order                   AdvSVO    SAdvVO    SVAdvO    SVOAdv
Adverb_Class Group                                               
Place        Beginner      0.086580  0.000000  0.051948  0.861472
             Intermediate  0.058252  0.019417  0.135922  0.786408
             Advanced      0.053571  0.017857  0.107143  0.821429
             Native        0.140625  0.007812  0.070312  0.781250
Extent       Beginner      0.021834  0.458515  0.034934  0.484716
             Intermediate  0.019231  0.625000  0.028846  0.326923
             Advanced      0.000000  0.732143  0.089286  0.178571
             Native        0.000000  0.664384  0.171233  0.164384
Manner       Beginner      0.000000  0.080357  0.084821  0.834821
             Intermediate  0.000000  0.192308  0.057692  0.750000
             Advanced      0.000000  0.017857  0.357143  0.625000
             Native        0.007353  0.095588  0.323529  0.573529
Quantity     Beginner      0.004329  0.177489  0.238095  0.580087
             Intermediate  0.009615  0.182692  0.230769  0.576923
             Advanced      0.000000  0.125000  0.357143  0.517857
             Native        0.000000  0.204225  0.507042  0.288732

In [11]:
writer.save()